# FireProx Phase 1 Demo - Asynchronous API

This notebook demonstrates all Phase 1 features of the FireProx asynchronous API.

**Prerequisites**: Firestore emulator must be running on port 8080.

**Note**: Jupyter notebooks require special handling for async code. Each async cell must use `await` and be preceded by cell magic if needed.

## 1. Setup and Initialization

In [1]:
from google.cloud import firestore
from fire_prox import AsyncFireProx, State

# Initialize native async Firestore client (assumes emulator is running)
client = firestore.AsyncClient(project='demo-project')
db = AsyncFireProx(client)
print("AsyncFireProx initialized successfully!")

AsyncFireProx initialized successfully!


## 2. Creating a New Document (DETACHED State)

In [2]:
# Get a collection reference
users = db.collection('users')

# Create a new document (not yet in Firestore)
user = users.new()
print(f"State: {user.state}")
print(f"Is detached: {user.is_detached()}")
print(f"Is dirty: {user.is_dirty()}")

State: DETACHED
Is detached: True
Is dirty: True


## 3. Setting Attributes on a DETACHED Document

In [3]:
# Set attributes using dot notation
user.name = 'Ada Lovelace'
user.year = 1815
user.occupation = 'Mathematician'

print(f"Name: {user.to_dict()['name']}")
print(f"Data: {user.to_dict()}")
print(f"Still detached: {user.is_detached()}")

Name: Ada Lovelace
Data: {'name': 'Ada Lovelace', 'year': 1815, 'occupation': 'Mathematician'}
Still detached: True


## 4. Saving with Custom ID (DETACHED → LOADED)

In [4]:
# Save with a custom document ID (async operation)
await user.save(doc_id='alovelace')

print(f"State after save: {user.state}")
print(f"Is loaded: {user.is_loaded()}")
print(f"Is dirty: {user.is_dirty()}")
print(f"Document ID: {user.id}")
print(f"Document path: {user.path}")

State after save: LOADED
Is loaded: True
Is dirty: False
Document ID: alovelace
Document path: users/alovelace


## 5. Getting a Document by Path (ATTACHED State)

In [5]:
# Get a document reference (doesn't fetch data yet)
user2 = db.doc('users/alovelace')

print(f"State: {user2.state}")
print(f"Is attached: {user2.is_attached()}")
print(f"Document ID: {user2.id}")
print(f"Document path: {user2.path}")
print("Data not fetched yet!")

State: ATTACHED
Is attached: True
Document ID: alovelace
Document path: users/alovelace
Data not fetched yet!


## 6. Explicit Fetch Required (No Lazy Loading in Async)

In [6]:
# Async API requires explicit fetch (no lazy loading)
# Attempting to access attributes before fetch raises error:
try:
    name = user2.name  # This will fail!
except AttributeError as e:
    print(f"Error (expected): {e}")
    print("\nAsync requires explicit fetch!")

Error (expected): Cannot access attribute 'name' on ATTACHED AsyncFireObject. Call await fetch() first to load data from Firestore.

Async requires explicit fetch!


In [7]:
# Must explicitly fetch data
await user2.fetch()

print(f"State after fetch: {user2.state}")
print(f"Is loaded: {user2.is_loaded()}")
print(f"Name: {user2.name}")
print(f"Full data: {user2.to_dict()}")

State after fetch: LOADED
Is loaded: True
Name: Ada Lovelace
Full data: {'occupation': 'Mathematician', 'name': 'Ada Lovelace', 'year': 1815}


## 7. Modifying a LOADED Document

In [8]:
# Modify attributes (synchronous operations)
user2.year = 1816
user2.contributions = ['Analytical Engine', 'First Algorithm']

print(f"Is dirty: {user2.is_dirty()}")
print(f"Modified year: {user2.year}")
print(f"Contributions: {user2.contributions}")

Is dirty: True
Modified year: 1816
Contributions: ['Analytical Engine', 'First Algorithm']


## 8. Saving Updates (Async)

In [9]:
# Save the modifications (async operation)
await user2.save()

print(f"Is dirty after save: {user2.is_dirty()}")
print(f"State: {user2.state}")
print("Changes saved to Firestore!")

Is dirty after save: False
State: LOADED
Changes saved to Firestore!


## 9. Refreshing Data with force=True

In [10]:
# Fetch fresh data from Firestore (async operation)
await user2.fetch(force=True)

print(f"Refreshed data: {user2.to_dict()}")
print(f"Year after refresh: {user2.year}")
print(f"Contributions: {user2.contributions}")

Refreshed data: {'occupation': 'Mathematician', 'contributions': ['Analytical Engine', 'First Algorithm'], 'name': 'Ada Lovelace', 'year': 1816}
Year after refresh: 1816
Contributions: ['Analytical Engine', 'First Algorithm']


## 10. Deleting Attributes

In [11]:
# Delete an attribute (synchronous)
del user2.contributions

print(f"Is dirty: {user2.is_dirty()}")
print(f"Data after delete: {user2.to_dict()}")
print("Attribute 'contributions' removed locally")

Is dirty: True
Data after delete: {'occupation': 'Mathematician', 'name': 'Ada Lovelace', 'year': 1816}
Attribute 'contributions' removed locally


In [12]:
# Save to persist the deletion (async)
await user2.save()
await user2.fetch(force=True)

print(f"Data after save: {user2.to_dict()}")
print("'contributions' field removed from Firestore")

Data after save: {'occupation': 'Mathematician', 'name': 'Ada Lovelace', 'year': 1816}
'contributions' field removed from Firestore


## 11. Creating Document with Auto-Generated ID

In [13]:
# Create a new document without specifying ID
user3 = users.new()
user3.name = 'Grace Hopper'
user3.year = 1906

# Save without doc_id - Firestore generates ID (async)
await user3.save()

print(f"Auto-generated ID: {user3.id}")
print(f"Path: {user3.path}")
print(f"Data: {user3.to_dict()}")

Auto-generated ID: KxWU5V2hDnM0i1tFsyAS
Path: users/KxWU5V2hDnM0i1tFsyAS
Data: {'name': 'Grace Hopper', 'year': 1906}


## 12. Collection Properties

In [14]:
# Inspect collection properties (synchronous)
print(f"Collection ID: {users.id}")
print(f"Collection path: {users.path}")
print(f"String repr: {str(users)}")
print(f"Repr: {repr(users)}")

Collection ID: users
Collection path: users
String repr: AsyncFireCollection(users)
Repr: <AsyncFireCollection path='users'>


## 13. Deleting a Document (LOADED → DELETED)

In [15]:
# Delete a document from Firestore (async operation)
await user3.delete()

print(f"State after delete: {user3.state}")
print(f"Is deleted: {user3.is_deleted()}")
print(f"ID still accessible: {user3.id}")
print(f"Path still accessible: {user3.path}")

State after delete: DELETED
Is deleted: True
ID still accessible: KxWU5V2hDnM0i1tFsyAS
Path still accessible: users/KxWU5V2hDnM0i1tFsyAS


## 14. Error Handling - Invalid Operations on DELETED

In [16]:
# Attempting operations on DELETED document raises errors
try:
    await user3.save()
except RuntimeError as e:
    print(f"Save error: {e}")

try:
    await user3.fetch()
except RuntimeError as e:
    print(f"Fetch error: {e}")

Save error: Cannot save() on a DELETED FireObject
Fetch error: Cannot fetch() on a DELETED FireObject


## 15. Hydration from Native Firestore Snapshot

In [17]:
# Use native async Firestore API to get a snapshot
from fire_prox import AsyncFireObject

doc_ref = client.collection('users').document('alovelace')
snapshot = await doc_ref.get()

# Hydrate to AsyncFireObject
user4 = AsyncFireObject.from_snapshot(snapshot)

print(f"State: {user4.state}")
print(f"Is loaded: {user4.is_loaded()}")
print(f"Data: {user4.to_dict()}")
print("Hydrated from native async snapshot!")

State: LOADED
Is loaded: True
Data: {'occupation': 'Mathematician', 'name': 'Ada Lovelace', 'year': 1816}
Hydrated from native async snapshot!


## 16. Working with Nested Data

In [18]:
# Create document with nested data structures
user5 = users.new()
user5.name = 'Alan Turing'
user5.address = {
    'city': 'London',
    'country': 'UK'
}
user5.achievements = ['Turing Machine', 'Enigma', 'Turing Test']

await user5.save(doc_id='aturing')
print(f"Nested data saved: {user5.to_dict()}")

Nested data saved: {'name': 'Alan Turing', 'address': {'city': 'London', 'country': 'UK'}, 'achievements': ['Turing Machine', 'Enigma', 'Turing Test']}


## 17. Accessing Nested Data

In [19]:
# Access nested fields
user6 = db.doc('users/aturing')
await user6.fetch()

print(f"City: {user6.address['city']}")
print(f"First achievement: {user6.achievements[0]}")
print(f"All achievements: {user6.achievements}")

City: London
First achievement: Turing Machine
All achievements: ['Turing Machine', 'Enigma', 'Turing Test']


## 18. Multiple Async Operations

In [20]:
# Create multiple documents with async operations
user7 = users.new()
user7.name = 'Katherine Johnson'
user7.year = 1918
await user7.save(doc_id='kjohnson')

user8 = users.new()
user8.name = 'Margaret Hamilton'
user8.year = 1936
await user8.save(doc_id='mhamilton')

print("Created multiple documents:")
print(f"  - {user7.name} ({user7.path})")
print(f"  - {user8.name} ({user8.path})")

Created multiple documents:
  - Katherine Johnson (users/kjohnson)
  - Margaret Hamilton (users/mhamilton)


## Summary

This demo covered all Phase 1 features of the **Async API**:

✅ **State Machine**: DETACHED → ATTACHED → LOADED → DELETED  
✅ **Dynamic Attributes**: Set/get/delete using dot notation (sync)  
✅ **Explicit Fetch**: **Must** use `await fetch()` (no lazy loading)  
✅ **Async Save**: `await save()` for create/update operations  
✅ **Async Delete**: `await delete()` to remove documents  
✅ **State Inspection**: `state`, `is_loaded()`, `is_dirty()`, etc. (sync)  
✅ **Collection Interface**: `new()`, `doc()`, properties (sync)  
✅ **Hydration**: `from_snapshot()` for native async query results  
✅ **Nested Data**: Dictionaries and lists as attributes  
✅ **Error Handling**: Clear messages for invalid operations  

### Key Differences: Async vs Sync

| Feature | Sync API | Async API |
|---------|----------|----------|
| Lazy Loading | ✅ Automatic | ❌ Not supported (Python limitation) |
| Fetch | `user.fetch()` or `user.name` | `await user.fetch()` (required) |
| Save | `user.save()` | `await user.save()` |
| Delete | `user.delete()` | `await user.delete()` |
| Attribute access | Triggers fetch if ATTACHED | Requires explicit `await fetch()` |

**Next**: See Phase 2 features (subcollections, queries, partial updates)